# **1. Mô tả ứng dụng** 

*   **Ứng dụng mà nhóm chọn:** huấn luyện Convolutional Neural Network để phân lớp ảnh bằng Stochastic Gradient Descent.
   *   **Input:** 1 bức ảnh số viết tay có kích thước 28x28.

   *   **Output:** số được viết trong bức ảnh.

   *   **Ý nghĩa của ứng dụng trong thực tế:** giúp máy tính có thể tự nhận biết trong hình là số mấy, từ đó tăng tốc độ cho các tác vụ liên quan như OCR...

*   **Ứng dụng này có cần phải tăng tốc không:** có, vì trong đó có phép convolution, phép nhân ma trận.

# **2. Cài đặt tuần tự**

*   **Thiết kế:** CNN sẽ có 1 convolutional layer, 1 max-pooling layer, 1 fully connected layer với activation là softmax.

    *   Trong quá trình lan truyền xuôi, trước tiên, ta sẽ chuẩn hoá giá trị của các pixel của hình ảnh về khoảng \[0,1], việc chuẩn hoá sẽ giúp ta tính toán dễ dàng hơn. Trong convolutional layer sẽ có 8 filter với kích thước là 3x3 được khởi tạo với các giá trị ngẫu nhiên nằm trong khoảng \[0,1], tương ứng với 8 node trong layer. Khi đi qua convolutional layer, ta sẽ được 8 ma trận có kích thước là 26x26 tương ứng với 8 filter. Trong max-pooling layer cũng sẽ có 8 node tương ứng với 8 ma trận vừa tìm được, kích thước của mỗi max-pooling là 2x2, max-pooling 2x2 sẽ giúp ta giảm kích thước ma trận xuống một nữa nhưng vẫn giữ được các đặc điểm quan trọng của bức ảnh, sau khi đi qua max-pooling layer, ta thu được 8 ma trận có kích thước là 13x13. Trong fully connected layer sẽ có 10 node tương ứng với 10 chữ số, mỗi node sẽ kết nối với tất cả các node trong max-pooling layer, vì vậy mỗi node sẽ có 13x13x8 = 1352 trọng số + với 10 bias, ta sẽ được 10 giá trị tương ứng với 10 node, sau đó ta sẽ tính giá trị softmax cho 10 giá trị này. Nhãn dự đoán của mô hình sẽ là nhãn có giá trị softmax cao nhất.

      ![maxpooling_prop](https://drive.google.com/uc?export=view&id=1cYlb4ye0m-3UM408og6SUW6_H1Y-xQ-B)

    *   Đối với giai đoạn lan truyền ngược trong fully connected layer, để đánh giá mô hình ta sẽ dùng cross-entropy loss: $L=-\text{ln}(p_c)$ với $p_c$ là xác suất được dự đoán cho nhãn đúng $c$, nên \begin{equation}\frac{\partial L}{\partial out_s(i)} = \begin{cases} 0 & \text{if } i \neq c\\  -\frac{1}{p_i} & \text{if } i = c\\ \end{cases}\end{equation} với $out_s(i)$ là giá trị softmax của nhãn $i$, vì vậy các node không phải nhãn đúng sẽ có gradient bằng 0. Ta sẽ tiếp tục tính \begin{equation}\frac{\partial out_s(k)}{\partial t} = \begin{cases} \frac{-e^{t_c}e^{t_k}}{S^2} & \text{if } k \neq c\\  \frac{e^{t_c}(S-e^{t_c})}{S^2} & \text{if } k = c\\ \end{cases}\end{equation} với $t$ là các giá trị trước khi đưa vào tính softmax, có dạng $t=w\*input+b$. Ta tính thêm $\frac{\partial t}{\partial w}=input$, $\frac{\partial t}{\partial b}=1$, $\frac{\partial t}{\partial input}=w$. Từ đó ta có thể áp dụng quy tắc mắc xích: $\frac{\partial L}{\partial w}=\frac{\partial L}{\partial out}\*\frac{\partial out}{\partial t}\*\frac{\partial t}{\partial w}$, $\frac{\partial L}{\partial b}=\frac{\partial L}{\partial out}\*\frac{\partial out}{\partial t}\*\frac{\partial t}{\partial b}$, $\frac{\partial L}{\partial input}=\frac{\partial L}{\partial out}\*\frac{\partial out}{\partial t}\*\frac{\partial t}{\partial input}$. Giờ ta có thể cập nhật trọng số và bias của fully connected layer bằng Stochastic Gradient Descent: $w_i=w_i-\alpha \frac{\partial L}{\partial w_i}$ với $w_i$ là trọng số thứ $i$ và $\alpha$ là hệ số học, $b_i=b_i-\alpha \frac{\partial L}{\partial b_i}$ với $b_i$ là bias thứ $i$ và $\alpha$ là hệ số học. Ta sẽ truyền ngược $\frac{\partial L}{\partial input}$ về cho maxpooling layer.

    *   Đối với giai đoạn lan truyền ngược trong maxpooling layer, ta sẽ phục hồi lại kích thước của ma trận trước khi truyền vào layer này và gán ngược trở lại các giá trị trong $\frac{\partial L}{\partial input}$ vào vị trí của giá trị lớn nhất tương ứng với kích thước của maxpooling, các vị trí còn lại sẽ bằng 0, ta sẽ truyền ngược ma trận này về cho convolutional layer:

      ![maxpooling_prop](https://drive.google.com/uc?export=view&id=1H_ugd98o7XR5-LfGk_rkPA_FEkCvBmk9)

    *   Đối với giai đoạn lan truyền ngược tron convolutional layer, ta có công thức $out(i,j)=\sum\limits_{x=0}^3\sum\limits_{y=0}^3 \text{image}(i+x,j+y)*\text{filter}(x,y)$, nên $\frac{\partial \text{out}(i,j)}{\partial \text{filter}(x,y)}=\text{image}(i+x,j+y)$, suy ra $\frac{\partial L}{\partial \text{filter}(x,y)}=\sum\limits_{i}^{}\sum\limits_{j}^{}\frac{\partial L}{\partial \text{out}(i,j)}\frac{\partial \text{out}(i,j)}{\partial \text{filter}(x,y)}$. Sau đó ta thực hiện việc cập nhật các trọng số của filter: $\text{filter}(x,y)=\text{filter}(x,y)-\alpha\frac{\partial L}{\partial \text{filter}(x,y)}$ với $\alpha$ là hệ số học.
    
*   **Đánh giá:**


# **3. Cài đặt song song (trên GPU)**

*   **Phân tích:**
*   **Thiết kế:**
*   **Đánh giá:**

# **4. Cài đặt song song (trên GPU) + tối ưu hóa**

# **5. Nhìn lại quá trình làm đồ án**

# **6. Tài liệu tham khảo**

https://victorzhou.com/blog/intro-to-cnns-part-1/

https://victorzhou.com/blog/intro-to-cnns-part-2/